In [12]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, VGG16, ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from kerastuner.tuners import RandomSearch
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2, VGG16, ResNet50
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model
from keras.optimizers import Adam
from kerastuner.tuners import RandomSearch
from keras.callbacks import EarlyStopping
import logging

# Data preparation and preprocessing

This section of code is focused on setting up the necessary environment and preprocessing for training a deep learning model.

In [7]:
SEED = 1865050  

random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Set up logging
logging.basicConfig(level=logging.INFO, filename='training.log', filemode='a', format='%(asctime)s - %(levelname)s - %(message)s')
logging.info("Starting the training script")

# Data Directories
TRAIN_DIR = "C:\\Users\\Pattarawadee\\OneDrive\\Desktop\\Master\\Year2\\Tri1\\DL\\HW\\2\\train"
TEST_DIR = "C:\\Users\\Pattarawadee\\OneDrive\\Desktop\\Master\\Year2\\Tri1\\DL\\HW\\2\\test"

# Data Augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

logging.info("Data augmentation setup complete.")

train_generator = datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Load test data
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

logging.debug("Training generator samples: %d", train_generator.samples)
logging.debug("Validation generator samples: %d", validation_generator.samples)
logging.debug("Testing generator samples: %d", test_generator.samples)

Found 4694 images belonging to 15 classes.
Found 1165 images belonging to 15 classes.
Found 1841 images belonging to 15 classes.


The results indicate that the dataset has been successfully loaded and partitioned into training, validation, and testing sets, with the images being categorized into 15 different classes. Specifically:

Training Set: 4694 images have been found and assigned to 15 different classes.
Validation Set: 1165 images are present, also distributed across 15 classes.
Testing Set: 1841 images have been identified, belonging to 15 classes as well.
This distribution ensures that the model has a variety of data to learn from (training set), a set to tune and validate its parameters (validation set), and a different set to evaluate its final performance (testing set), all across the same 15 classes.

# Models

This section of the code defines the functionality for setting up three different convolutional neural network (CNN) models for image classification, as well as a mechanism to prevent overfitting during training.

Early Stopping: An EarlyStopping callback is defined to monitor the validation loss during training. If the validation loss stops improving for 3 consecutive epochs, the training will be halted, and the model weights from the epoch with the best validation loss will be restored. This helps in preventing overfitting and ensures that the model does not waste computational resources once it ceases to learn.

Model Building Functions: Three separate functions are defined to build CNN models based on different architectures, specifically MobileNetV2 (build_mobilenet_model), VGG16 (build_vgg_model), and ResNet50 (build_resnet_model). Each function follows a similar structure:

Base Model: The base convolutional layer of the respective architecture is loaded with pre-trained weights from ImageNet, and its weights are set to be non-trainable. This is a form of transfer learning where the model benefits from features learned on a large dataset.

Additional Layers: On top of the base model, additional layers are added including Flatten, Dense, BatchNormalisation, Activation, Dropout, and a final Dense layer. These layers are meant to adapt the learned features to the specific task at hand.

Regularisation and Dropout: L2 regularisation is applied to the Dense layer and Dropout is used to prevent overfitting.

Output Layer: The final Dense layer has 15 units (matching the number of classes in the dataset) and uses softmax activation to produce a probability distribution over the classes.

Compilation: The model is compiled with Adam optimiser, categorical crossentropy as the loss function, and accuracy as a metric for evaluation.

In [16]:
# Stop training when a monitored quantity has stopped improving.
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

def build_mobilenet_model():
    base_model = tf.keras.applications.MobileNetV2(weights='imagenet', input_shape=(224, 224, 3), include_top=False)
    base_model.trainable = False

    model = models.Sequential([
        base_model,
        layers.Flatten(),
        layers.Dense(512, kernel_regularizer=l2(1e-3)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.5),
        layers.Dense(15),
        layers.BatchNormalization(),
        layers.Activation('softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def build_vgg_model():
    base_model = tf.keras.applications.VGG16(weights='imagenet', input_shape=(224, 224, 3), include_top=False)
    base_model.trainable = False

    model = models.Sequential([
        base_model,
        layers.Flatten(),
        layers.Dense(512, kernel_regularizer=l2(1e-3)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.5),
        layers.Dense(15),
        layers.BatchNormalization(),
        layers.Activation('softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def build_resnet_model():
    base_model = tf.keras.applications.ResNet50(weights='imagenet', input_shape=(224, 224, 3), include_top=False)
    base_model.trainable = False

    model = models.Sequential([
        base_model,
        layers.Flatten(),
        layers.Dense(512, kernel_regularizer=l2(1e-3)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.5),
        layers.Dense(15),
        layers.BatchNormalization(),
        layers.Activation('softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Model Training

This section of the code involves training three different deep learning models based on MobileNetV2, VGG16, and ResNet50 architectures for image classification.

### MobileNetV2

In [17]:
# Train MobileNetV2 Model
# Building the MobileNetV2 model
mnV2_model = build_mobilenet_model()

# Setting up a checkpoint to save the best version of the model based on validation loss
mnV2_cp = ModelCheckpoint(filepath='best_model_mobilenet.h5', save_best_only=True, monitor='val_loss', mode='min', verbose=1)

# Configuring TensorBoard for logging training statistics in the 'mobilenet' directory
mnV2_tb = TensorBoard(log_dir=os.path.join(os.getcwd(), 'logs/mobilenet'))

# Training the MobileNetV2 model with early stopping, model checkpoint, and TensorBoard callbacks
mnV2_history = mnV2_model.fit(train_generator, validation_data=validation_generator, epochs=20, callbacks=[early_stopping, mnV2_cp, mnV2_tb])


Epoch 1/20
147/147 [==============================] - ETA: 0s - loss: 2.5570 - accuracy: 0.5750
Epoch 1: val_loss improved from inf to 1.96617, saving model to best_model_mobilenet.h5


C:\Users\Pattarawadee\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


147/147 [==============================] - 248s 2s/step - loss: 2.5570 - accuracy: 0.5750 - val_loss: 1.9662 - val_accuracy: 0.7433
Epoch 2/20
147/147 [==============================] - ETA: 0s - loss: 2.1893 - accuracy: 0.7137
Epoch 2: val_loss did not improve from 1.96617
147/147 [==============================] - 193s 1s/step - loss: 2.1893 - accuracy: 0.7137 - val_loss: 1.9718 - val_accuracy: 0.7605
Epoch 3/20
147/147 [==============================] - ETA: 0s - loss: 2.0750 - accuracy: 0.7367
Epoch 3: val_loss improved from 1.96617 to 1.94858, saving model to best_model_mobilenet.h5
147/147 [==============================] - 191s 1s/step - loss: 2.0750 - accuracy: 0.7367 - val_loss: 1.9486 - val_accuracy: 0.7717
Epoch 4/20
147/147 [==============================] - ETA: 0s - loss: 1.9727 - accuracy: 0.7588
Epoch 4: val_loss improved from 1.94858 to 1.88180, saving model to best_model_mobilenet.h5
147/147 [==============================] - 191s 1s/step - loss: 1.9727 - accuracy: 0.

### VGG16

In [18]:
# Train VGG16 Model
# Building the VGG16 model
vgg16_model = build_vgg_model()

# Setting up a checkpoint to save the best version of the model based on validation loss
vgg16_cp = ModelCheckpoint(filepath='best_model_vgg.h5', save_best_only=True, monitor='val_loss', mode='min', verbose=1)

# Configuring TensorBoard for logging training statistics in the 'vgg' directory
vgg16_tb = TensorBoard(log_dir=os.path.join(os.getcwd(), 'logs/vgg'))

# Training the VGG16 model with early stopping, model checkpoint, and TensorBoard callbacks
vgg16_history = vgg16_model.fit(train_generator, validation_data=validation_generator, epochs=20, callbacks=[early_stopping, vgg16_cp, vgg16_tb])


Epoch 1/20
147/147 [==============================] - ETA: 0s - loss: 3.0819 - accuracy: 0.3530
Epoch 1: val_loss improved from inf to 2.65903, saving model to best_model_vgg.h5
147/147 [==============================] - 725s 5s/step - loss: 3.0819 - accuracy: 0.3530 - val_loss: 2.6590 - val_accuracy: 0.5262
Epoch 2/20
147/147 [==============================] - ETA: 0s - loss: 2.6176 - accuracy: 0.5096
Epoch 2: val_loss improved from 2.65903 to 2.38065, saving model to best_model_vgg.h5
147/147 [==============================] - 717s 5s/step - loss: 2.6176 - accuracy: 0.5096 - val_loss: 2.3806 - val_accuracy: 0.5906
Epoch 3/20
147/147 [==============================] - ETA: 0s - loss: 2.4567 - accuracy: 0.5616
Epoch 3: val_loss improved from 2.38065 to 2.29273, saving model to best_model_vgg.h5
147/147 [==============================] - 730s 5s/step - loss: 2.4567 - accuracy: 0.5616 - val_loss: 2.2927 - val_accuracy: 0.6300
Epoch 4/20
147/147 [==============================] - ETA: 0s 

### ResNet50

In [19]:
# Train ResNet50 Model
# Building the ResNet50 model
res50_model = build_resnet_model()

# Setting up a checkpoint to save the best version of the model based on validation loss
res50_cp = ModelCheckpoint(filepath='best_model_resnet.h5', save_best_only=True, monitor='val_loss', mode='min', verbose=1)

# Configuring TensorBoard for logging training statistics in the 'resnet' directory
res50_tb = TensorBoard(log_dir=os.path.join(os.getcwd(), 'logs/resnet'))

# Training the ResNet50 model with early stopping, model checkpoint, and TensorBoard callbacks
res50_history = res50_model.fit(train_generator, validation_data=validation_generator, epochs=20, callbacks=[early_stopping, res50_cp, res50_tb])


Epoch 1/20
147/147 [==============================] - ETA: 0s - loss: 3.0241 - accuracy: 0.1525
Epoch 1: val_loss improved from inf to 3.56068, saving model to best_model_resnet.h5
147/147 [==============================] - 445s 3s/step - loss: 3.0241 - accuracy: 0.1525 - val_loss: 3.5607 - val_accuracy: 0.1227
Epoch 2/20
147/147 [==============================] - ETA: 0s - loss: 2.7154 - accuracy: 0.1939
Epoch 2: val_loss improved from 3.56068 to 2.83089, saving model to best_model_resnet.h5
147/147 [==============================] - 444s 3s/step - loss: 2.7154 - accuracy: 0.1939 - val_loss: 2.8309 - val_accuracy: 0.1725
Epoch 3/20
147/147 [==============================] - ETA: 0s - loss: 2.6386 - accuracy: 0.2118
Epoch 3: val_loss improved from 2.83089 to 2.75416, saving model to best_model_resnet.h5
147/147 [==============================] - 435s 3s/step - loss: 2.6386 - accuracy: 0.2118 - val_loss: 2.7542 - val_accuracy: 0.2017
Epoch 4/20
147/147 [==============================] -

# Visualising Training Metrics with TensorBoard

This section is dedicated to utilising TensorBoard, a comprehensive visualisation tool from TensorFlow's ecosystem, to monitor and assess the training metrics of deep learning models.

In [20]:
# Enabling TensorBoard within Jupyter Notebooks
%load_ext tensorboard

# Starting TensorBoard and specifying the directory where the logs are stored
%tensorboard --logdir logs


# Model Evaluation on Test Data

In this section, the pre-trained models from different architectural backgrounds, specifically VGG, ResNet, and MobileNetV2, are thoroughly evaluated on the test dataset to understand their performance in testing data set.

In [21]:
# Defining the architecture names using abbreviations
arch_names = ['VGG', 'ResNet', 'MobileNetV2']

# Initializing a dictionary to hold the loaded models
loaded_models = {}

# Loading the pre-trained models for each architecture
# 'best_model_vgg.h5', 'best_model_resnet.h5', 'best_model_mobilenet.h5'
for architecture in arch_names:
    model_filepath = f'best_model_{architecture.lower()}.h5'
    loaded_models[architecture] = load_model(model_filepath)

# Evaluate each loaded model on the test dataset and store the accuracy in a dictionary
test_accuracies = {}

# Iterating through the loaded models for evaluation
for architecture, loaded_model in loaded_models.items():
    _, accuracy = loaded_model.evaluate(test_generator, verbose=0)
    test_accuracies[architecture] = accuracy

# Displaying the accuracy results for each architecture
test_accuracies


{'vgg': 0.7327539324760437,
 'resnet': 0.2927756607532501,
 'mobilenet': 0.8413905501365662}

The performance evaluation of the VGG, ResNet, and MobileNetV2 models on the test dataset has yielded the following accuracy results:

VGG: 73.28%
ResNet: 29.28%
MobileNetV2: 84.14%


These results indicate that the MobileNetV2 model outperformed the other two models by a significant margin. On the other hand, the ResNet model showed a surprisingly low accuracy, which might be indicative of issues during training or a mismatch in the problem complexity and model capacity. It could also be due to the model not being fine-tuned adequately for the given task. The VGG model achieved a respectable accuracy, showcasing its strong feature extraction capabilities, though it could not surpass the MobileNetV2.